In [ ]:
from detection.detection import anonymize_on_video, cascade_detect_image
from existing.viola.viola import viola_anonymize_on_video, viola_detect_on_img
from existing.mtcnn.mtcnn import mtcnn_anonymize_on_video, mtcnn_detect_on_img
from existing.ssd.ssd import ssd_detect_on_video, ssd_detect_on_img

# Detection on image

Viola

In [ ]:
file1 = 'WIDER_train\images/6--Funeral/6_Funeral_Funeral_6_14.jpg'
file2 = 'WIDER_train\images/35--Basketball/35_Basketball_Basketball_35_98.jpg'
test_files = [file1, file2]

for i in range(2):
    viola_detect_on_img(test_files[i], True, f"test_images/test_viola{i+1}.jpg")

In [ ]:
for i in range(2):
    mtcnn_detect_on_img(test_files[i], True, f"test_images/test_mtcnn{i+1}.jpg")

In [ ]:
for i in range(2):
    ssd_detect_on_img(test_files[i], True, f"test_images/test_ssd{i+1}.jpg")

In [ ]:
for i in range(2):
    cascade_detect_image(test_files[i], True, f"test_images/test_cascade{i+1}.jpg")

# Calculating metrics

Getting test set

In [ ]:
import random
import os
import cv2

# Define the path to the validation set and annotations
val_path = 'WIDER_val'
annotations_path = 'wider_face_split/wider_face_val_bbx_gt.txt'

# Create a list of file paths for the images and annotations
image_paths = []
annotation_boxes = []
current_image_path = ""
with open(annotations_path, 'r') as file:
    for line in file:
        line = line.strip()
        if line.endswith('.jpg'):
            current_image_path = os.path.join(val_path, 'images', line)
            image_paths.append(current_image_path)
        else:
            if line != "":
                num_of_bbx = int(line)
                current_annotation_boxes = []
                for i in range(num_of_bbx):
                    next_line = file.readline()
                    x1, y1, w, h, _, _, _, _, _, _ = list(map(int, next_line.strip().split()))
                    current_annotation_boxes.append((x1, y1, x1+w, y1+h))
                annotation_boxes.append(current_annotation_boxes)

random_indices = random.sample(range(len(image_paths)), 100)
sample_image_paths = [image_paths[i] for i in random_indices]
sample_annotation_boxes = [annotation_boxes[i] for i in random_indices]

# Metrics for Viola

In [ ]:
# Load the Viola-Jones cascade classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Define some variables to keep track of the counts
total_faces = 0
detected_faces = 0
false_negatives = 0
false_positives = 0

# Iterate over all the images and annotations
for i in range(len(sample_image_paths)):
    # Read the image
    image = cv2.imread(sample_image_paths[i])
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Get the bounding boxes for all faces in the image
    total_faces += len(sample_annotation_boxes[i])

    # Detect faces in the image using the classifier
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    detected_faces += len(faces)

        # Compare the detections with the annotation boxes
    for (x, y, w, h) in faces:
        matched = False
        for (xmin, ymin, xmax, ymax) in sample_annotation_boxes[i]:
            if (xmin <= x <= xmax) and (ymin <= y <= ymax):
                matched = True
                break
        if not matched:
            false_positives += 1

    # Count false negatives
    for (xmin, ymin, xmax, ymax) in sample_annotation_boxes[i]:
        matched = False
        for (x, y, w, h) in faces:
            if (xmin <= x <= xmax) and (ymin <= y <= ymax):
                matched = True
                break
        if not matched:
            false_negatives += 1

# Compute the false negative ratio
false_negative_ratio = false_negatives / total_faces
print('False negative ratio for Viola-Jones:', false_negative_ratio)

# Compute the false positive ratio
false_positive_ratio = false_positives / total_faces
print('False positive ratio for Viola-Jones:', false_positive_ratio)

# Metrics for our cascade

In [ ]:
from model.model import predict
from tensorflow import keras
model = keras.models.load_model('best_model.h5', compile=False)

# Load the Viola-Jones cascade classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Define some variables to keep track of the counts
total_faces = 0
detected_faces = 0
cnn_approved_faces = 0
false_negatives = 0
false_positives = 0

# Iterate over all the images and annotations
for i in range(len(sample_image_paths)):
    # Read the image
    image = cv2.imread(sample_image_paths[i])
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Get the bounding boxes for all faces in the image
    total_faces += len(sample_annotation_boxes[i])

    # Detect faces in the image using the classifier
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    detected_faces += len(faces)
    cnn_faces = []

    for (x, y, w, h) in faces:
        result = predict(model, gray[y:y+h, x:x+w])
        if result == 1:
            cnn_approved_faces += 1
            cnn_faces.append((x, y, w, h))

        # Compare the detections with the annotation boxes
    for (x, y, w, h) in cnn_faces:
        matched = False
        for (xmin, ymin, xmax, ymax) in sample_annotation_boxes[i]:
            if (xmin <= x <= xmax) and (ymin <= y <= ymax):
                matched = True
                break
        if not matched:
            false_positives += 1

    # Count false negatives
    for (xmin, ymin, xmax, ymax) in sample_annotation_boxes[i]:
        matched = False
        for (x, y, w, h) in cnn_faces:
            if (xmin <= x <= xmax) and (ymin <= y <= ymax):
                matched = True
                break
        if not matched:
            false_negatives += 1

# Compute the false negative ratio
false_negative_ratio = false_negatives / total_faces
print('False negative ratio for Viola-CNN Cascade:', false_negative_ratio)

# Compute the false positive ratio
false_positive_ratio = false_positives / total_faces
print('False positive ratio for Viola-CNN Cascade:', false_positive_ratio)